In [12]:
import numpy as np
import pandas as pd
from pytorch_metric_learning import miners, losses, distances 
import os
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler
import pickle
import time
from sklearn.model_selection import train_test_split, KFold
import json
from tqdm.auto import tqdm
import random 
import math 
import faiss # pip install faiss-gpu 
from rank_bm25 import BM25Okapi
import pickle 

In [13]:
data = []

with open("law_candidates_60K.jsonl") as f:
    for line in f:
        data.append(json.loads(line))

queries, passages, answers = [], [], []
for i in tqdm(range(len(data)), position=0, leave=True):
    query = data[i]["summary"]
    passage = data[i]["text"]
    answer = data[i]["answer"]
    queries.append(query)
    passages.append(passage)
    answers.append(answer)

all_data = pd.DataFrame(list(zip(queries, passages, answers)), columns=["queries", "passages", "answers"])


  0%|          | 0/60069 [00:00<?, ?it/s]

In [21]:
model_name = "monologg/kobigbird-bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

loading file https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/00ac7c2886f9d4555133877badce522b93b38439d90b0135d9b414cc1fafd167.34d17d2d06e0d29acc69761e3ddeced0dfdcf4cefa0aa81a1bb267a7dfdd5bcb
loading file https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/e2eb4ad30139b806997f999b45c0a0d9ea38b14e0d97f42db852be137e061b1e.616843352d77fff459e989408eaacf1280dc39dcd346ff746aa3b3fbe6a123d9
loading file https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/monologg/kobigbird-bert-base/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/9bea998b48658e35dd618115a266f6c173183a9a4261fc6e40730d74c4b67899.e3640e465e51ce85d94923a0b396029ecc2e3e4c7764031eee57ab272637652d
loading file https://huggingface.co/monologg/kobigbird-b

In [17]:
train_df, test_df = train_test_split(all_data, test_size=0.2, random_state=42)
valid_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

query_index_dict = {} 

for i in range(len(queries)):
    query_index_dict[queries[i]] = [] 

for i in range(len(queries)): 
    query_index_dict[queries[i]].append(i) 

# create train set and validation set 

construct hard negative pairs

In [65]:
# create BM25 instance 
tokenized_corpus = []

for candidate in tqdm(passages, position=0, leave=True): 
    tokenized_corpus.append(tokenizer.tokenize(candidate)) 

bm25 = BM25Okapi(tokenized_corpus)

  0%|          | 0/60069 [00:00<?, ?it/s]

In [125]:
train_positive_dict, train_negative_dict = {}, {} 
train_queries = train_df["queries"].values 

for query in tqdm(train_queries, position=0, leave=True): 
    train_positive_dict[query] = [] 
    train_negative_dict[query] = [] 


for query in tqdm(train_queries[:100], position=0, leave=True, desc="creating train dataset"): 
    cnt = 0 
    for idx in query_index_dict[query]:
        try: 
            train_positive_dict[query].append(passages[idx]) 
            cnt += 1 
        except:
            continue 
    tokenized_query = tokenizer.tokenize(query) 
    negative_code_scores = bm25.get_scores(tokenized_query) 
    negative_code_ranking = negative_code_scores.argsort()[::-1]
    idxs = query_index_dict[query] 
    negative_cnt = 0 
    for i in range(len(negative_code_ranking)): 
        if negative_code_ranking[i] not in idxs: 
            train_negative_dict[query].append(passages[negative_code_ranking[i]])  
            negative_cnt += 1 
        if negative_cnt >= cnt: 
            break     

  0%|          | 0/48055 [00:00<?, ?it/s]

creating train dataset:   0%|          | 0/100 [00:00<?, ?it/s]

In [132]:
valid_positive_dict, valid_negative_dict = {}, {} 
valid_queries = valid_df["queries"].values 

for query in tqdm(valid_queries, position=0, leave=True): 
    valid_positive_dict[query] = [] 
    valid_negative_dict[query] = [] 


for query in tqdm(valid_queries[:100], position=0, leave=True, desc="creating valid dataset"): 
    cnt = 0 
    for idx in query_index_dict[query]: 
        try: 
            valid_positive_dict[query].append(passages[idx])  
            cnt += 1 
        except:
            continue 
    tokenized_query = tokenizer.tokenize(query) 
    negative_code_scores = bm25.get_scores(tokenized_query) 
    negative_code_ranking = negative_code_scores.argsort()[::-1]
    idxs = query_index_dict[query] 
    negative_cnt = 0
    for i in range(len(negative_code_ranking)):
        if negative_code_ranking[i] not in idxs: 
            valid_negative_dict[query].append(passages[negative_code_ranking[i]]) 
            negative_cnt += 1 
        if negative_cnt >= cnt:
            break 

  0%|          | 0/6007 [00:00<?, ?it/s]

creating valid dataset:   0%|          | 0/100 [00:00<?, ?it/s]

In [149]:
query

'가격담합에 관한 수회의 합의 중에 일시적으로 가격인하 등의 조치가 있는 경우, 합의가 파기되거나 종료되어 합의가 단절된 것으로 볼 수 있는지 여부'

In [147]:
valid_negative_dict[query]

['구 독점규제 및 공정거래에 관한 법률(2004. 12. 31. 법률 제7315호로 개정되기 전의 것) 제19조 제1항 제1호에 정한 가격 결정 등의 합의 및 그에 터잡은 실행행위가 있었던 경우 부당한 공동행위가 종료한 날은 그 합의에 터잡은 실행행위가 종료한 날이므로, 합의에 참가한 일부 사업자가 부당한 공동행위를 종료하기 위해서는 다른 사업자에 대하여 합의에서 탈퇴하였음을 알리는 명시적 내지 묵시적인 의사표시를 하고 독자적인 판단에 따라 담합이 없었더라면 존재하였을 가격 수준으로 인하하는 등 합의에 반하는 행위를 하여야 한다. 또한, 합의에 참가한 사업자 전부에 대하여 부당한 공동행위가 종료되었다고 하기 위해서는 합의에 참가한 사업자들이 명시적으로 합의를 파기하고 각 사업자가 각자의 독자적인 판단에 따라 담합이 없었더라면 존재하였을 가격 수준으로 인하하는 등 합의에 반하는 행위를 하거나 또는 합의에 참가한 사업자들 사이에 반복적인 가격 경쟁 등을 통하여 담합이 사실상 파기되었다고 인정할 수 있을 만한 행위가 일정 기간 계속되는 등 합의가 사실상 파기되었다고 볼 수 있을 만한 사정이 있어야 한다.']

In [146]:
valid_positive_dict[query] 

['일반적으로 가격담합의 경우, 수회의 합의 중에 일시적으로 사업자들의 가격인하 등의 조치가 있더라도 사업자들의 명시적인 담합파기 의사표시가 있었음이 인정되지 않는 이상 합의가 파기되거나 종료되어 합의가 단절되었다고 보기 어렵다.']

In [123]:
with open("train_positive_pairs.pkl", "wb") as file: 
    pickle.dump(train_positive_pairs, file) 
    
with open("train_negative_pairs.pkl", "wb") as file: 
    pickle.dump(train_negative_pairs, file) 

with open("valid_positive_pairs.pkl", "wb") as file:
    pickle.dump(valid_positive_pairs, file)
    
with open("valid_negative_pairs.pkl", "wb") as file: 
    pickle.dump(valid_negative_pairs, file) 

In [157]:
with open("valid_negative_dict.pkl", "rb") as file: 
    d = pickle.load(file) 